In [ ]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

In [ ]:
from Bio.PDB import PDBParser
import numpy as np

def measure_bond_distance(
    pdb_file, 
    chain_id_1, 
    res_id_1, 
    res_name_1,
    atom_name_1, 
    chain_id_2, 
    res_id_2, 
    res_name_2,
    atom_name_2):
    """
    Measure the bond distance between two atoms in a PDB file.

    Args:
        pdb_file (str): Path to the PDB file.
        chain_id_1 (str): Chain ID where the first atom is located.
        res_id_1 (tuple): Tuple of (residue sequence number, insertion code) for the first atom.
        atom_name_1 (str): Name of the first atom.
        chain_id_2 (str): Chain ID where the second atom is located.
        res_id_2 (tuple): Tuple of (residue sequence number, insertion code) for the second atom.
        atom_name_2 (str): Name of the second atom.

    Returns:
        float: Distance between the two atoms in angstroms.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)

    # Locate the first atom
    chain = structure[0][chain_id]
    residue_1 = chain[res_id_1]
    atom_1 = residue_1[atom_name_1]

    # Locate the second atom
    residue_2 = chain[res_id_2]
    atom_2 = residue_2[atom_name_2]

    # Calculate the distance
    distance = np.linalg.norm(atom_1.coord - atom_2.coord)
    return distance

# # Example usage
# pdb_file = "example.pdb"
# chain_id = "A"
# res_id_1 = (10, " ")  # Residue 10, no insertion code
# atom_name_1 = "CA"  # Alpha carbon
# res_id_2 = (20, " ")  # Residue 20, no insertion code
# atom_name_2 = "CB"  # Beta carbon

# bond_distance = measure_bond_distance(pdb_file, chain_id, res_id_1, atom_name_1, res_id_2, atom_name_2)
# print(f"Bond distance: {bond_distance:.2f} Å")
